In [1]:
import pandas as pd
import configparser
from datetime import datetime
import os
import numpy as np
from pyspark.sql.functions import sum,avg,max,min,mean,count
from schemas import dict_schemas, dict_numeric_columns
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.functions import udf, col, to_date, date_format
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, monotonically_increasing_id
from pyspark.sql.types import StructType as R, StructField as Fld, DoubleType as Dbl,StringType as Str, IntegerType as Int, DateType as Date, TimestampType as TS

In [2]:
def create_spark_session():
    """ 
    Create and return a Spark Session 
    """
    
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    
    return spark

In [3]:
spark = create_spark_session()

In [4]:
csv_tables = ['champs', 'maps', 'matches', 'participants', 'queues', 'seasons', 'stats', 'teambans', 'teamstats']

In [10]:
output_data = r'data_source/parquet_files'

In [6]:
def check_rows_csv_to_parquet(csv_df, output_data, table, csv_rows):
    
    pqt = spark.read.parquet(f"{output_data}/{table}")
    
    pqt_rows = pqt.count()
        
    if pqt_rows == csv_rows:
        print( 'Ok !' )
    else:
        print( f'CSV has: {csv_rows} rows, while PQT has: {pqt_rows}' )

In [9]:
for table in csv_tables:
    
    input_data_csv = r'Riot Matches/{}.csv'.format(table)
    print(f"path: {input_data_csv}")
    
    spark_df = spark.read.options(header='true').csv(input_data_csv, schema = dict_schemas[table])
    
    csv_rows = spark_df.count()
#     spark_df = pd.read_csv(input_data_csv)

#     spark_df = treat_numeric_cols(spark_df, table)
    
#     spark_df = spark.createDataFrame(spark_df, schema = dict_schemas[table])
    
    spark_df = spark_df.dropna(how = 'all')
    
    spark_df.write.parquet(f"{output_data}/{table}")
    
    check_rows_csv_to_parquet(spark_df, output_data, table, csv_rows)
    
    print(f"{table} load success")

path: Riot Matches/champs.csv
Ok !
champs load success
path: Riot Matches/maps.csv
Ok !
maps load success
path: Riot Matches/matches.csv
Ok !
matches load success
path: Riot Matches/participants.csv
Ok !
participants load success
path: Riot Matches/queues.csv
Ok !
queues load success
path: Riot Matches/seasons.csv
Ok !
seasons load success
path: Riot Matches/stats.csv
Ok !
stats load success
path: Riot Matches/teambans.csv
Ok !
teambans load success
path: Riot Matches/teamstats.csv
Ok !
teamstats load success


In [53]:
stats = spark.read.parquet(f"{output_data}/stats")

In [55]:
stats.printSchema()

root
 |-- id: integer (nullable = true)
 |-- win: integer (nullable = true)
 |-- item1: integer (nullable = true)
 |-- item2: integer (nullable = true)
 |-- item3: integer (nullable = true)
 |-- item4: integer (nullable = true)
 |-- item5: integer (nullable = true)
 |-- item6: integer (nullable = true)
 |-- trinket: integer (nullable = true)
 |-- kills: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- largestkillingspree: integer (nullable = true)
 |-- largestmultikill: integer (nullable = true)
 |-- killingsprees: integer (nullable = true)
 |-- longesttimespentliving: integer (nullable = true)
 |-- doublekills: integer (nullable = true)
 |-- triplekills: integer (nullable = true)
 |-- quadrakills: integer (nullable = true)
 |-- pentakills: integer (nullable = true)
 |-- legendarykills: integer (nullable = true)
 |-- totdmgdealt: integer (nullable = true)
 |-- magicdmgdealt: integer (nullable = true)
 |-- physicaldmgdealt: in

In [56]:
champs = spark.read.parquet(f"{output_data}/champs")

champs.printSchema()

root
 |-- name: string (nullable = true)
 |-- id: integer (nullable = true)



In [57]:
participants = spark.read.parquet(f"{output_data}/participants")

participants.printSchema()

root
 |-- id: integer (nullable = true)
 |-- matchid: integer (nullable = true)
 |-- player: integer (nullable = true)
 |-- championid: integer (nullable = true)
 |-- ss1: integer (nullable = true)
 |-- ss2: integer (nullable = true)
 |-- role: string (nullable = true)
 |-- position: string (nullable = true)



In [58]:
matches = spark.read.parquet(f"{output_data}/matches")

matches.printSchema()

root
 |-- id: integer (nullable = true)
 |-- gameid: string (nullable = true)
 |-- platformid: string (nullable = true)
 |-- queueid: integer (nullable = true)
 |-- seasonid: integer (nullable = true)
 |-- duration: double (nullable = true)
 |-- creation: double (nullable = true)
 |-- version: string (nullable = true)



In [15]:
participants.count()

1834520

In [16]:
stats.count()

1834517

# Creating Participants Stats per Match

In [17]:
participants = participants.withColumnRenamed("id","participants_id")

In [18]:
stats = stats.withColumnRenamed("id","stats_id")

In [19]:
champs = champs.withColumnRenamed("id","champs_id")

In [20]:
matches = matches.withColumnRenamed("id","match_id")

In [21]:
player_df = participants.join(stats, ([stats.stats_id == participants.participants_id]), "left")

In [22]:
player_df = player_df.join(champs, ([player_df.championid == champs.champs_id]), "left")

In [23]:
player_df = player_df.join(matches, ([player_df.matchid == matches.match_id]), "left")

In [24]:
def check_df_is_not_empty(df):
    
    if df.count() > 0:
        print('Check OK !')
    else:
        raise ValueError('DataFrame empty after operation')

In [25]:
player_df = player_df.dropDuplicates()

In [26]:
check_df_is_not_empty(player_df)

Check OK !


In [27]:
player_df.select('creation').show(3)

+-----------------+
|         creation|
+-----------------+
|1.494764416274E12|
|1.493818672404E12|
|1.493811885648E12|
+-----------------+
only showing top 3 rows



In [28]:
def create_time_columns(df, ts_col):
    
    get_timestamp = udf(lambda x: datetime.fromtimestamp(x / 1000), TS())

    # create date column from original timestamp ts_col column
    df = df.withColumn("create_match_date", get_timestamp(col(ts_col)))
    
    # extract time detailed time columns
    df = df.withColumn("create_match_hour", hour("create_match_date"))
    df = df.withColumn("create_match_day", dayofmonth("create_match_date"))
    df = df.withColumn("create_match_month", month("create_match_date"))
    df = df.withColumn("create_match_year", year("create_match_date"))
    
    return df

In [29]:
player_df = create_time_columns(player_df, 'creation')

In [30]:
player_df.printSchema()

root
 |-- participants_id: integer (nullable = true)
 |-- matchid: integer (nullable = true)
 |-- player: integer (nullable = true)
 |-- championid: integer (nullable = true)
 |-- ss1: integer (nullable = true)
 |-- ss2: integer (nullable = true)
 |-- role: string (nullable = true)
 |-- position: string (nullable = true)
 |-- stats_id: integer (nullable = true)
 |-- win: integer (nullable = true)
 |-- item1: integer (nullable = true)
 |-- item2: integer (nullable = true)
 |-- item3: integer (nullable = true)
 |-- item4: integer (nullable = true)
 |-- item5: integer (nullable = true)
 |-- item6: integer (nullable = true)
 |-- trinket: integer (nullable = true)
 |-- kills: integer (nullable = true)
 |-- deaths: integer (nullable = true)
 |-- assists: integer (nullable = true)
 |-- largestkillingspree: integer (nullable = true)
 |-- largestmultikill: integer (nullable = true)
 |-- killingsprees: integer (nullable = true)
 |-- longesttimespentliving: integer (nullable = true)
 |-- doubleki

In [30]:
#player_df.write.partitionBy("create_match_year", "create_match_month").parquet(f"{output_data}/players_statistics_per_match/")

# Creating Player Stats per Match

In [59]:
teamstats = spark.read.parquet(f"{output_data}/teamstats")

teamstats.printSchema()

teamstats = teamstats.withColumnRenamed("matchid","team_stats_match_id")
#teambans = spark.read.parquet(f"{output_data}/teambans.parquet")

root
 |-- matchid: integer (nullable = true)
 |-- teamid: integer (nullable = true)
 |-- firstblood: integer (nullable = true)
 |-- firsttower: integer (nullable = true)
 |-- firstinhib: integer (nullable = true)
 |-- firstbaron: integer (nullable = true)
 |-- firstdragon: integer (nullable = true)
 |-- firstharry: integer (nullable = true)
 |-- towerkills: integer (nullable = true)
 |-- inhibkills: integer (nullable = true)
 |-- baronkills: integer (nullable = true)
 |-- dragonkills: integer (nullable = true)
 |-- harrykills: integer (nullable = true)



In [60]:
queues = spark.read.parquet(f"{output_data}/queues")

queues.printSchema()

queues = queues.withColumnRenamed("queueId","queue_id")

root
 |-- description: string (nullable = true)
 |-- map: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- queueId: integer (nullable = true)



In [33]:
teams_df = participants.join(stats, ([participants.participants_id == stats.stats_id]), "left")

In [34]:
#teams_df = teams_df.join(matches, ([teams_df.matchid == matches.match_id]), "left")

In [35]:
# create side team column from player column
get_side_team = udf(lambda x: 'Blue' if x <= 5 else 'Red', Str())

teams_df = teams_df.withColumn("team_side", get_side_team(col("player")))

In [36]:
count_players = teams_df.groupBy("matchid").count().withColumnRenamed('matchid', 'count_match_id')

In [37]:
teams_df = teams_df.join(count_players, ([teams_df.matchid == count_players.count_match_id]), "left")

In [38]:
teams_df = teams_df.filter(teams_df['count'] == 10)

In [39]:
#teams_df.filter(teams_df['duration'].isNotNull()).count()

In [40]:
teams_df.count()

1825280

In [41]:
teams_df = teams_df.groupBy('team_side', 'matchid', 'win').agg(\
    sum('kills').alias('total_kills'), \
    sum('deaths').alias('total_deaths'), \
    sum('assists').alias('total_assists'), \
    sum('doublekills').alias('total_doublekills'), \
    sum('triplekills').alias('total_triplekills'), \
    sum('quadrakills').alias('total_quadrakills'), \
    sum('pentakills').alias('total_pentakills'), \
    sum('totdmgdealt').alias('total_totdmgdealt'), \
    sum('magicdmgdealt').alias('total_magicdmgdealt'), \
    sum('physicaldmgdealt').alias('total_physicaldmgdealt'), \
    sum('truedmgdealt').alias('total_truedmgdealt'), \
    sum('largestcrit').alias('total_largestcrit'), \
    sum('totdmgtochamp').alias('total_totdmgtochamp'), \
    sum('magicdmgtochamp').alias('total_magicdmgtochamp'), \
    sum('physdmgtochamp').alias('total_physdmgtochamp'), \
    sum('truedmgtochamp').alias('total_truedmgtochamp'), \
    sum('totheal').alias('total_totheal'), \
    sum('totunitshealed').alias('total_totunitshealed'), \
    sum('dmgselfmit').alias('total_dmgselfmit'), \
    sum('dmgtoobj').alias('total_dmgtoobj'), \
    sum('dmgtoturrets').alias('total_dmgtoturrets'), \
    sum('visionscore').alias('total_visionscore'), \
    sum('goldearned').alias('total_goldearned'), \
    sum('goldspent').alias('total_goldspent'), \
    sum('totminionskilled').alias('total_totminionskilled'), \
    sum('neutralminionskilled').alias('total_neutralminionskilled'), \
    sum('pinksbought').alias('total_pinksbought'), \
    sum('wardsbought').alias('total_wardsbought'), \
    sum('wardsplaced').alias('total_wardsplaced'), \
    sum('wardskilled').alias('total_wardskilled'))

In [42]:
teams_df = teams_df.join(matches, ([teams_df.matchid == matches.match_id]), "left")

In [43]:
teams_df = teams_df.join(queues, ([teams_df.queueid == queues.queue_id]), "left")

In [44]:
# transform teamid to BLue(100) or Red(200) to join dataframes
get_side_team_by_id = udf(lambda x: 'Blue' if int(x) == 100 else 'Red', Str())

teamstats = teamstats.withColumn("side_team", get_side_team_by_id(col("teamid")))

In [45]:
teams_df = teams_df.join(teamstats, ([teams_df.matchid == teamstats.team_stats_match_id,
                                     teams_df.team_side == teamstats.side_team]), "left")

In [46]:
teams_df.filter(teams_df['creation'].isNotNull()).count()

365057

In [47]:
# create match_date column from original timestamp creation column
get_timestamp = udf(lambda x: datetime.fromtimestamp(x / 1000), TS())
teams_df = teams_df.withColumn("create_match_date", get_timestamp(col("creation")))

# extract columns to create time table
teams_df = teams_df.withColumn("create_match_hour", hour("create_match_date"))
teams_df = teams_df.withColumn("create_match_day", dayofmonth("create_match_date"))
teams_df = teams_df.withColumn("create_match_month", month("create_match_date"))
teams_df = teams_df.withColumn("create_match_year", year("create_match_date"))

In [48]:
teams_df.select('create_match_year').distinct().show()

+-----------------+
|create_match_year|
+-----------------+
|             2015|
|             2014|
|             2016|
|             2017|
+-----------------+



In [49]:
#teams_df.write.partitionBy("create_match_year", "create_match_month").parquet(f"{output_data}/players_statistics_per_match/")

In [50]:
teams_df = teams_df.drop('side_team', 'match_id', 'queue_id', 'teamid')

In [52]:
teams_df.printSchema()

root
 |-- team_side: string (nullable = true)
 |-- matchid: integer (nullable = true)
 |-- win: integer (nullable = true)
 |-- total_kills: long (nullable = true)
 |-- total_deaths: long (nullable = true)
 |-- total_assists: long (nullable = true)
 |-- total_doublekills: long (nullable = true)
 |-- total_triplekills: long (nullable = true)
 |-- total_quadrakills: long (nullable = true)
 |-- total_pentakills: long (nullable = true)
 |-- total_totdmgdealt: long (nullable = true)
 |-- total_magicdmgdealt: long (nullable = true)
 |-- total_physicaldmgdealt: long (nullable = true)
 |-- total_truedmgdealt: long (nullable = true)
 |-- total_largestcrit: long (nullable = true)
 |-- total_totdmgtochamp: long (nullable = true)
 |-- total_magicdmgtochamp: long (nullable = true)
 |-- total_physdmgtochamp: long (nullable = true)
 |-- total_truedmgtochamp: long (nullable = true)
 |-- total_totheal: long (nullable = true)
 |-- total_totunitshealed: long (nullable = true)
 |-- total_dmgselfmit: long (

In [51]:
output_data

'data_source/parquet_files'

In [69]:
teams_df.select('duration', 'creation', 'team_side', 'matchid',  'win', 'gameid').printSchema()

root
 |-- duration: long (nullable = true)
 |-- creation: long (nullable = true)
 |-- team_side: string (nullable = true)
 |-- matchid: long (nullable = true)
 |-- win: long (nullable = true)
 |-- gameid: long (nullable = true)



In [70]:
teams_df.filter(teams_df['duration'].isNotNull()).select('duration', 'creation', 'team_side','matchid','win', 'gameid').show(5)

+--------+-------------+---------+-------+---+----------+
|duration|     creation|team_side|matchid|win|    gameid|
+--------+-------------+---------+-------+---+----------+
|    2045|1494521072459|      Red|     26|  0|3179228168|
|    2045|1494521072459|     Blue|     26|  1|3179228168|
|    2557|1494510773735|     Blue|     29|  1|3179074023|
|    2557|1494510773735|      Red|     29|  0|3179074023|
|    2427|1483835559537|      Red|    474|  0|3008614130|
+--------+-------------+---------+-------+---+----------+
only showing top 5 rows



In [ ]:
duration, creation, team_side, participants_id, matchid, championid, win, gameid

In [36]:
agg_columns = ['kills', 'deaths', 'assists', 'doublekills', 'triplekills', 'quadrakills', 'pentakills',
                'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt', 'truedmgdealt', 'largestcrit', 'totdmgtochamp',
                'magicdmgtochamp', 'physdmgtochamp', 'truedmgtochamp', 'totheal', 'totunitshealed', 'dmgselfmit',
                'dmgtoobj', 'dmgtoturrets', 'visionscore', 'goldearned', 'goldspent', 'totminionskilled', 
                'neutralminionskilled', 'pinksbought', 'wardsbought', 'wardsplaced', 'wardskilled']

#df.groupBy(df("age")).agg(first("name"), first("some other col to includel"), ...)

In [157]:
teams_df.groupBy('matchid', 'gameid', 'team_side', 'win').count().show(5)

+-------+----------+---------+---+-----+
|matchid|    gameid|team_side|win|count|
+-------+----------+---------+---+-----+
|   1580|2132065859|     Blue|  1|    5|
|   1580|2132065859|      Red|  0|    5|
|   1591|1964812748|     Blue|  0|    5|
|   1591|1964812748|      Red|  1|    5|
|   1645|1932609680|     Blue|  1|    5|
+-------+----------+---------+---+-----+
only showing top 5 rows



In [132]:
counts = teams_df.groupBy("matchid").count().withColumn
teams_df.alias("df").join(counts, col("df.matchid") == col("counts.matchid")

teams_df.join(
    broadcast(counts), teams_df.matchid == counts.matchid).show()

teams_df.join(counts, ([counts.matchid == teams_df.matchid]), "left")

+---------------+-------+------+----------+---+---+-----------+--------+--------+---+-----+-----+-----+-----+-----+-----+-------+-----+------+-------+-------------------+----------------+-------------+----------------------+-----------+-----------+-----------+----------+--------------+-----------+-------------+----------------+------------+-----------+-------------+---------------+--------------+--------------+-------+--------------+----------+--------+------------+-----------+------+-----------+-------------+------------+------------+----------+---------+-----------+----------+----------------+--------------------+--------------+----------------+--------------+--------+-----------+-----------+-----------+-----------+----------+--------+------+----------+-------+--------+--------+--------+-------+---------+-------+-----+
|participants_id|matchid|player|championid|ss1|ss2|       role|position|stats_id|win|item1|item2|item3|item4|item5|item6|trinket|kills|deaths|assists|largestkillingspr

In [97]:
teams_df.groupBy('matchid', 'team_side').count().show

DataFrame[matchid: int, team_side: string, count: bigint]

In [ ]:
teams_df['team'] = df['player'].apply(lambda x: '1' if x <= 5 else '2')

In [82]:
teams_df.groupBy('matchid').sum()

DataFrame[matchid: int, sum(participants_id): bigint, sum(matchid): bigint, sum(player): bigint, sum(championid): bigint, sum(ss1): bigint, sum(ss2): bigint, sum(stats_id): bigint, sum(win): bigint, sum(item1): bigint, sum(item2): bigint, sum(item3): bigint, sum(item4): bigint, sum(item5): bigint, sum(item6): bigint, sum(trinket): bigint, sum(kills): bigint, sum(deaths): bigint, sum(assists): bigint, sum(largestkillingspree): bigint, sum(largestmultikill): bigint, sum(killingsprees): bigint, sum(longesttimespentliving): bigint, sum(doublekills): bigint, sum(triplekills): bigint, sum(quadrakills): bigint, sum(pentakills): bigint, sum(legendarykills): bigint, sum(totdmgdealt): bigint, sum(magicdmgdealt): bigint, sum(physicaldmgdealt): bigint, sum(truedmgdealt): bigint, sum(largestcrit): bigint, sum(totdmgtochamp): bigint, sum(magicdmgtochamp): bigint, sum(physdmgtochamp): bigint, sum(truedmgtochamp): bigint, sum(totheal): bigint, sum(totunitshealed): bigint, sum(dmgselfmit): bigint, sum(

In [ ]:
stats.groupBy('')

# Analysis